# Installation

In [ ]:
# pip install langchain langchain_community langchain_openai beautifulsoup4 python-dotenv langchainhub

In [19]:
!nvidia-smi

Sat Feb  8 14:58:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.03                 Driver Version: 566.03         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060      WDDM  |   00000000:01:00.0  On |                  N/A |
| 44%   49C    P0             47W /  170W |    1399MiB /  12288MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:
pip install faiss-cpu faiss-gpu-cu12

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement faiss-gpu-cu12 (from versions: none)
ERROR: No matching distribution found for faiss-gpu-cu12


# AI Agent Development

**Loading the environment variables (from .env file):**

- USER_AGENT
- OPENAI_API_KEY
- TAVILY_API_KEY

In [1]:
from dotenv import load_dotenv

load_dotenv()                   # Loading API key from .env file

True

## A) Tools

### 1.Retriever Tool

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain.tools.retriever import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [3]:
loader = WebBaseLoader("https://www.dasa.org")
docs = loader.load()

print(docs[0].metadata['source'])
print(docs[0].metadata['title'])
print(docs[0].metadata['description'])

https://www.dasa.org
DASA
Transform Your Enterprise Into a High-Performance Digital Organization Tools and expertise to create agile, resilient software teams that drive lasting digital


In [4]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
print("# Documents:",len(documents))

embeddings = OpenAIEmbeddings()
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()


retriever_tool = create_retriever_tool (retriever, "DASA_search", "Search for information about DASA. For any questions about DASA, you must use this tool")

# Documents: 2


### 2. Web Search Tool

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults()

## B) AI Agent

In [ ]:
tools = [retriever_tool, search_tool]

**Prompt Template:**

- LangChain Hub allows access to predefined prompts, chains, and agents.
- The pulled prompt template designed for OpenAI's function calling capabilities. It is typically formatted for use with OpenAI GPT models in an agent-based setup

In [7]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt

c:\Users\Admin\Desktop\AIPoint Projects\.venv\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='